# Cross Validation
We've looked at a range of topics involved with fitting a model to data. This began with the simplest of regression cases and determining criteria for an optimal model, which led us to mean squarred error. From there, we further examined overfitting and underfitting which motivated train test split and later, the bias variance tradeoff. Here, we synthesize many of these ideas into a new sampling, optimization meta-routine known as cross validation. 

A common form of cross validation is known as K-folds. In this process, the dataset is partitioned into K equally sized groups. Each group is then used as a hold out test set while the remaining k-1 groups are used as a training set. This then produces K different models, one for each of the hold out test sets. These models can then be averaged (perhaps a weighted average based on their test set performance) in order to produce a finalized model.

This is also a very useful method for helping to determine the generalization of our models, or the anticipated difference between train and test errors for the model.

## 1. K-Folds
Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

In [1]:
import pandas as pd

In [2]:

def kfolds(data, k):
    
    data = pd.DataFrame(data)
    num_observations = len(data)
    fold_size = num_observations//k
    num_leftovers = num_observations%k
    folds = []
    start_obs = 0
    
    for fold_n in range(1, k+1):
        if fold_n <= num_leftovers:
            fold = data.iloc[start_obs : start_obs + fold_size + 1]
            folds.append(fold)
            start_obs += fold_size+1
        else:
            fold = data.iloc[start_obs : start_obs + fold_size]
            folds.append(fold)
            start_obs += fold_size
            
    return folds #folds should be a list of subsets of data

## 2. Cross Validation
* Split your dataset into 10 groups using your kfolds function above.
* Perform linear regression on each and calculate the training and test error. 
* Create a simple bar chart to display the various train and test errors for each of the 10 folds.

In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [4]:
df = pd.read_excel('movie_data_detailed_with_ols.xlsx')
X_feats = ['budget', 'imdbRating',
       'Metascore', 'imdbVotes']
y_feat = 'domgross'
df.head()

,budget,domgross,title,Response_Json,Year,imdbRating,Metascore,imdbVotes,Model
0,13000000,25682380,21 &amp; Over,0,2008,6.8,48,206513,4.912759e+07
1,45658735,13414714,Dredd 3D,0,2012,0.0,0,0,2.267265e+05
2,20000000,53107035,12 Years a Slave,0,2013,8.1,96,537525,1.626624e+08
3,61000000,75612460,2 Guns,0,2013,6.7,55,173726,7.723381e+07
4,40000000,95020213,42,0,2013,7.5,62,74170,4.151958e+07


In [5]:
folds = kfolds(df, k=10)

# Previews, just to demonstrate

In [6]:
folds[0]

,budget,domgross,title,Response_Json,Year,imdbRating,Metascore,imdbVotes,Model
0,13000000,25682380,21 &amp; Over,0,2008,6.8,48,206513,4.912759e+07
1,45658735,13414714,Dredd 3D,0,2012,0.0,0,0,2.267265e+05
2,20000000,53107035,12 Years a Slave,0,2013,8.1,96,537525,1.626624e+08


In [7]:
folds[1]

,budget,domgross,title,Response_Json,Year,imdbRating,Metascore,imdbVotes,Model
3,61000000,75612460,2 Guns,0,2013,6.7,55,173726,7.723381e+07
4,40000000,95020213,42,0,2013,7.5,62,74170,4.151958e+07
5,225000000,38362475,47 Ronin,0,2013,6.3,28,128766,1.605898e+08


In [8]:
folds[8]

,budget,domgross,title,Response_Json,Year,imdbRating,Metascore,imdbVotes,Model
24,17000000,54239856,Evil Dead,0,2013,6.5,57,139940,4.076999e+07
25,160000000,238679850,Fast and Furious 6,0,0,0.0,0,0,2.383743e+08
26,150000000,393050114,Frozen,0,2013,7.5,74,483555,2.242330e+08


In [9]:
folds[9]

,budget,domgross,title,Response_Json,Year,imdbRating,Metascore,imdbVotes,Model
27,140000000,122523060,G.I. Joe: Retaliation,0,2013,5.8,41,158210,1.193156e+08
28,60000000,46000903,Gangster Squad,0,2013,6.7,40,188846,7.125032e+07
29,30000000,4167493,Gloria,0,1980,7.1,0,0,-1.783223e+07


In [10]:
def mse(residual_col):
    residual_col = pd.Series(residual_col)
    return None

In [13]:
test_errs = []
train_errs = []
k=10

for n in range(k):
    #Split into the train and test sets for this fold
    train = pd.concat([fold for i, fold in enumerate(folds) if i != n])
    test = folds[n]
    #Fit Linear Regression Model
    ols = LinearRegression()
    ols.fit(train[X_feats], train[y_feat])
    #Evaluate Train and Test Errors
    y_hat_train = ols.predict(train[X_feats])
    y_hat_test = ols.predict(test[X_feats])
    

#Plot Train Versus Test Errors for each of the 10 folds

[ 8.36914025e+07  2.67462291e+07  1.43522381e+08  1.06803778e+08
  6.67309587e+07  1.33050697e+07  1.33779142e+08  1.71634816e+08
  2.12912270e+07  4.57326715e+07  6.50109817e+07  1.56402468e+08
  6.33048738e+07  6.18184507e+07  1.43782154e+08  7.57482641e+07
  1.99073959e+08  1.07654033e+08  7.14951328e+07  1.36549591e+07
  9.44208501e+07  5.12896770e+07  1.34472862e+08  2.50499810e+08
  1.24658891e+08  7.96643928e+07 -3.96782659e+07]
[ 2.34589684e+07  5.69843196e+06  1.21209468e+08  1.12632295e+08
  3.16649086e+07 -7.47279492e+06  1.65393870e+08  1.19545986e+08
  2.15385782e+07  5.19241912e+07  4.17222517e+07  1.29252180e+08
  5.66922378e+07  9.30909645e+07  1.36639546e+08  2.52116936e+07
  2.04832844e+08  9.11363286e+07  1.20855308e+08  1.87158590e+07
  9.84478571e+07  2.25141698e+07  1.57530653e+08  2.71010253e+08
  1.79897416e+08  7.78223705e+07 -2.12456851e+07]
[ 4.79626813e+07  4.35820445e+07  1.57501069e+08  8.03341036e+07
  3.67243806e+07  1.78313107e+08  1.33084225e+08  1.390

## 4. Analysis
What do you notice about the train and test errors?

In [ ]:
#Your answer here

## 5. Shuffling Datasets
Write a function to randomly sort your dataset prior to cross validation.  
Why  might you want to do this?

In [ ]:
#Your function here

In [ ]:
#Your answer here